## 必要なモジュールのインストール

In [ ]:
import sys
import os
sys.path.append(os.pardir)
import numpy as np
import pickle
import pandas as pd
import re
from gensim.models import word2vec
import copy
from tqdm import tqdm_notebook
from utils import label_to_one_hot

## 学習/テストデータの読み込み

In [ ]:
with open('../data/train_data.pickle', 'rb') as rb:
    train_data = pickle.load(rb)

with open('../data/test_data.pickle', 'rb') as rb:
    test_data = pickle.load(rb)

### データ確認

In [ ]:
df_train = pd.DataFrame(train_data, columns=['label', 'text'])
df_test = pd.DataFrame(test_data, columns=['label', 'text'])

print('train_data')
display(df_train.head())
display(df_train['label'].value_counts())
print()
print('test_data')
display(df_test.head())
display(df_test['label'].value_counts())

### ラベルとテキストに分割

In [ ]:
x_train_text = [x[1] for x in train_data]
y_train = [x[0] for x in train_data]
x_test_text = [x[1] for x in test_data]
y_test = [x[0] for x in test_data]

## テキストをベクトルに変換する
今回は、word2vecモデルを使用してテキストを構成する単語をベクトルに変換し、
その平均値をとる。  
学習データ(orテストデータ)のi番目のラベルの文書全体を$ \textit{D}_{i} $とし、$ \textit{D}_{i} $が$ k $個の単語で構成されているとすると
$$
\textit{D}_{i} : {x_{1}, x_{2},... x_{k}} 
$$
となるので、$x_{j} \in D_{i}$をword2vecに変換したベクトルを$\boldsymbol{x}_{j}$とすると$ \textit{D}_{i} $の文書ベクトルを
$$
\frac{1}{k} \sum^{k}_{i=1} \boldsymbol{x}_{i}
$$
とする。$\boldsymbol{x}_{j}$が存在しない単語については除外する。

In [ ]:
#model = word2vec.Word2Vec.load("yagi/word2vec.gensim.model")
model = word2vec.Word2Vec.load('../data/basic_words_model.model')
def text_to_vector(text_list):
    text_vectors = []
    for text in text_list:
        words = re.split(r'( +)|\n', text)
        words = [x for x in words if x]
        for word in words:
            try:
                if word :
                    vector = model[word]
                    text_vectors.append(copy.copy(vector))
            except KeyError as f:
                pass
    text_vectors = np.array(text_vectors)
    return np.average(text_vectors, axis=0)
    

In [ ]:
x_train = np.array([text_to_vector(x) for x in tqdm_notebook(x_train_text)])
x_test = np.array([text_to_vector(x) for x in tqdm_notebook(x_test_text)])
print(x_train.shape)
print(x_test.shape)

## ラベルをIDに置き換える
今回分析対象の委員会を以下の通り通し番号をつける。

|委員会 |番号|
|---|---|
|本会議|0|
|内閣委員会|1|
|総務委員会|2|
|法務委員会|3|
|外務委員会|4|
|財務金融委員会|5|
|文部科学委員会|6|
|厚生労働委員会|7|
|農林水産委員会|8|
|経済産業委員会|9|
|国土交通委員会|10|
|環境委員会|11|
|安全保障委員会|12|
|予算委員会|13|
|決算行政監視委員会|14|
|議院運営委員会|15|



In [ ]:
classify_dict = {'本会議':0, '内閣委員会':1, '総務委員会':2, '法務委員会':3, '外務委員会':4, '財務金融委員会':5, '文部科学委員会':6, '厚生労働委員会':7, '農林水産委員会':8, \
                '経済産業委員会':9, '国土交通委員会':10, '環境委員会':11, '安全保障委員会':12, '予算委員会':13, '決算行政監視委員会':14, '議院運営委員会':15}
classify_size = len(classify_dict)
y_train = np.array([classify_dict[x] for x in tqdm_notebook(y_train)])
y_test = np.array([classify_dict[x] for x in tqdm_notebook(y_test)])

y_train

## クラスの重みを算出する
分析対象の委員会のデータ数のばらつきが大きいため、少ないクラス(委員会)の学習時のコストを大きくする。

In [ ]:
labels = np.arange(16)
cost_weight = np.zeros((16))
for label in labels:
    cnt = np.sum(y_train==label)
    cost_weight[label] = cnt

max_cnt = np.max(cost_weight)
cost_weight= np.reciprocal(cost_weight) * max_cnt
cost_weight = {int(label):cost_weight[label] for label in labels}
cost_weight

### One-hotベクトルの作成
上記通し番号に沿って、one-hotベクトルに変換する。以下に法務委員会の場合のone-hotベクトルを示す。
$$
\left(\begin{array}{c}
            0 \\
            0 \\
            0 \\
            1 \\
            0 \\
            ... \\
            0 \\
        \end{array}\right) \quad
$$

In [ ]:
classify_size = len(classify_dict)
y_train = np.array([label_to_one_hot(classify_size, x) for x in tqdm_notebook(y_train)])
y_test = np.array([label_to_one_hot(classify_size ,x) for x in tqdm_notebook(y_test)])

In [ ]:
y_train

In [ ]:
with open('train_data.pickle', 'wb') as wb:
    pickle.dump((x_train, y_train), wb)
    
with open('test_data.pickle', 'wb') as wb:
    pickle.dump((x_test, y_test), wb)
    
with open('cost_weight.pickle', 'wb') as wb:
    pickle.dump(cost_weight, wb)